Afficher les Chunks par tranche de 10000

In [1]:
import pandas as pd

# Lire le fichier CSV en morceaux de 10 000 lignes
chunks = pd.read_csv('en.openfoodfacts.org.products.csv', encoding="UTF-8", sep="\t", on_bad_lines='skip', chunksize=10000, low_memory=False)

displayed_lines = []

# Compteur pour afficher chaque chunk
chunk_count = 1

# Parcourir chaque chunk
for chunk in chunks:
    displayed_lines.append(chunk)
    
    # Afficher les 10 premières lignes de chaque chunk
    print(f"Chunk {chunk_count} :")
    print(chunk.head(10))
    print("\n")  # Ligne vide pour une meilleure lisibilité
    chunk_count += 1

# Vérifier le nombre total de chunks
print(f"Nombre total de chunks : {len(displayed_lines)}")

Chunk 1 :
   code                                                url  \
0     1  http://world-en.openfoodfacts.org/product/0000...   
1     2  http://world-en.openfoodfacts.org/product/0000...   
2     3  http://world-en.openfoodfacts.org/product/0000...   
3     4  http://world-en.openfoodfacts.org/product/0000...   
4     5  http://world-en.openfoodfacts.org/product/0000...   
5     6  http://world-en.openfoodfacts.org/product/0000...   
6     7  http://world-en.openfoodfacts.org/product/0000...   
7     8  http://world-en.openfoodfacts.org/product/0000...   
8     9  http://world-en.openfoodfacts.org/product/0000...   
9     9  http://world-en.openfoodfacts.org/product/0000...   

                      creator   created_t      created_datetime  \
0                         inf  1634745456  2021-10-20T15:57:36Z   
1                     kiliweb  1722606455  2024-08-02T13:47:35Z   
2                  prepperapp  1716818343  2024-05-27T13:59:03Z   
3                      elcoco  15601764

Nettoyer nutriscore_grade

In [2]:
import pandas as pd

# Concaténer tous les morceaux en un seul DataFrame
df = pd.concat(displayed_lines, ignore_index=True)

# Nettoyer la colonne 'nutriscore_grade'
valid_grades = ['a', 'b', 'c', 'd', 'e']
df_cleaned = df[df['nutriscore_grade'].isin(valid_grades)]

# Exporter le DataFrame nettoyé dans un fichier CSV
df_cleaned.to_csv('nettoyage_nutriscore_grade.csv', index=False, encoding='UTF-8')

# Charger le fichier nettoyage_nutriscore_grade.csv et afficher les 10 premières lignes de la colonne nutriscore_grade
df_nutriscore = pd.read_csv('nettoyage_nutriscore_grade.csv', encoding='UTF-8')
print(df_nutriscore['nutriscore_grade'].head(10))

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_6428\4024455562.py:14: DtypeWarning: Columns (0,11,17,31,32,33,35,51,67,72) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nutriscore = pd.read_csv('nettoyage_nutriscore_grade.csv', encoding='UTF-8')


0    d
1    e
2    a
3    e
4    d
5    d
6    d
7    d
8    d
9    d
Name: nutriscore_grade, dtype: object


supprimer les colonnes vides à partir d’un certain pourcentage de 70%

In [3]:
import pandas as pd

# Charger le fichier nettoyé
df = pd.read_csv('nettoyage_nutriscore_grade.csv', encoding='UTF-8')

# Déterminer le seuil pour garder une colonne (30 % des lignes doivent être non-nulles)
threshold = 0.3 * len(df)

# Liste des colonnes à conserver (product_name et brands)
columns_to_keep = ['product_name', 'brands']

# Supprimer les colonnes qui ont au moins 70 % de NaN, mais conserver product_name et brands
df_cleaned = df.drop(columns=[col for col in df.columns if col not in columns_to_keep and df[col].isnull().sum() > (0.7 * len(df))])

# Exporter le DataFrame nettoyé dans un fichier CSV appelé pourcentage.csv
df_cleaned.to_csv('pourcentage.csv', index=False, encoding='UTF-8')

# Optionnel : Afficher les 10 premières lignes du DataFrame nettoyé
print(df_cleaned.head(10))

# Sélection des colonnes 'product_name' et 'brands'
selected_data = df[['product_name', 'brands']]

# Affichage des données sélectionnées
print(selected_data)

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_6428\209147431.py:4: DtypeWarning: Columns (0,11,17,31,32,33,35,51,67,72) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('nettoyage_nutriscore_grade.csv', encoding='UTF-8')


  code                                                url  \
0    1  http://world-en.openfoodfacts.org/product/0000...   
1    4  http://world-en.openfoodfacts.org/product/0000...   
2    5  http://world-en.openfoodfacts.org/product/0000...   
3    6  http://world-en.openfoodfacts.org/product/0000...   
4    8  http://world-en.openfoodfacts.org/product/0000...   
5   10  http://world-en.openfoodfacts.org/product/0000...   
6   13  http://world-en.openfoodfacts.org/product/0000...   
7   15  http://world-en.openfoodfacts.org/product/0000...   
8   20  http://world-en.openfoodfacts.org/product/0000...   
9   23  http://world-en.openfoodfacts.org/product/0000...   

                      creator   created_t      created_datetime  \
0                         inf  1634745456  2021-10-20T15:57:36Z   
1                      elcoco  1560176426  2019-06-10T14:20:26Z   
2                   touchette  1605337720  2020-11-14T07:08:40Z   
3                     rigione  1541625386  2018-11-07T21:16:

supprimer les lignes où tous les nutriments sont vides
Problème

In [4]:
import pandas as pd

# Charger le fichier CSV
df = pd.read_csv('pourcentage.csv')

# Liste des colonnes à supprimer
colonnes_a_supprimer = [
    'carnitine_100g', 'sulphate_100g', 'nitrate_100g', 
    'acidity_100g', 'choline_100g', 'phylloquinone_100g', 
    'beta-glucan_100g', 'inositol_100g', 'glycemic-index_100g', 
    'water-hardness_100g'
]

# Vérifier les colonnes existantes dans le DataFrame
print("Colonnes existantes dans le DataFrame :", df.columns)

# Identifier les colonnes qui existent dans le DataFrame
colonnes_existantes = [col for col in colonnes_a_supprimer if col in df.columns]

# Supprimer les colonnes existantes
df = df.drop(columns=colonnes_existantes)

# Sauvegarder les modifications dans un nouveau fichier CSV
df.to_csv('nutriments_vides.csv', index=False)

# Afficher les 10 premières lignes du DataFrame nettoyé
print(df.head(10))

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_6428\2344678205.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('pourcentage.csv')


Colonnes existantes dans le DataFrame : Index(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'last_modified_by',
       'last_updated_t', 'last_updated_datetime', 'product_name', 'quantity',
       'brands', 'brands_tags', 'categories', 'categories_tags',
       'categories_en', 'labels', 'labels_tags', 'labels_en', 'countries',
       'countries_tags', 'countries_en', 'ingredients_text',
       'ingredients_tags', 'ingredients_analysis_tags', 'serving_size',
       'serving_quantity', 'additives_n', 'additives_tags', 'additives_en',
       'nutriscore_score', 'nutriscore_grade', 'nova_group', 'pnns_groups_1',
       'pnns_groups_2', 'food_groups', 'food_groups_tags', 'food_groups_en',
       'states', 'states_tags', 'states_en', 'ecoscore_score',
       'ecoscore_grade', 'nutrient_levels_tags', 'product_quantity',
       'unique_scans_n', 'popularity_tags', 'completeness', 'last_image_t',
       'last_image_datetime', 'm

suppressions des doublons sur marque et produit

In [5]:
import pandas as pd

# Charger le fichier CSV
df = pd.read_csv('nutriments_vides.csv')

# Trouver les doublons dans les colonnes 'brands' et 'product_name'
duplicates = df[df.duplicated(subset=['brands', 'product_name'], keep=False)]

# Afficher les doublons
print(duplicates[['brands', 'product_name']])

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_6428\3228162036.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('nutriments_vides.csv')


            brands             product_name
5            Bijou        Madeleines nature
7            Bijou     Madeleines ChocoLait
13           Bijou                      NaN
16       Herbalife                      NaN
17           Bijou          Fondants Citron
...            ...                      ...
1156017        NaN         Pain au Chocolat
1156020        NaN        Miel de printemps
1156023        NaN           Saucisse fumée
1156029        NaN              Riz au lait
1156030        NaN  Mousse au chocolat noir

[237536 rows x 2 columns]


In [6]:
import pandas as pd

# Charger le fichier CSV
df = pd.read_csv('nutriments_vides.csv')

# Supprimer les doublons basés sur les colonnes 'brands' et 'product_name'
df_cleaned = df.drop_duplicates(subset=['brands', 'product_name'], keep='first')

# Afficher les 10 premières lignes du DataFrame nettoyé
print(df_cleaned.head(10))

# Si vous souhaitez sauvegarder le DataFrame nettoyé dans un nouveau fichier CSV
df_cleaned.to_csv('nutriments_vides_cleaned.csv', index=False)

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_6428\1751967287.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('nutriments_vides.csv')


  code                                                url  \
0    1  http://world-en.openfoodfacts.org/product/0000...   
1    4  http://world-en.openfoodfacts.org/product/0000...   
2    5  http://world-en.openfoodfacts.org/product/0000...   
3    6  http://world-en.openfoodfacts.org/product/0000...   
4    8  http://world-en.openfoodfacts.org/product/0000...   
5   10  http://world-en.openfoodfacts.org/product/0000...   
6   13  http://world-en.openfoodfacts.org/product/0000...   
7   15  http://world-en.openfoodfacts.org/product/0000...   
8   20  http://world-en.openfoodfacts.org/product/0000...   
9   23  http://world-en.openfoodfacts.org/product/0000...   

                      creator   created_t      created_datetime  \
0                         inf  1634745456  2021-10-20T15:57:36Z   
1                      elcoco  1560176426  2019-06-10T14:20:26Z   
2                   touchette  1605337720  2020-11-14T07:08:40Z   
3                     rigione  1541625386  2018-11-07T21:16:

suppressions des variables redondantes (_tags , _en)

In [8]:
import pandas as pd

# Charger le fichier CSV nettoyé
df_cleaned = pd.read_csv('nutriments_vides_cleaned.csv')

# Identifier les colonnes redondantes se terminant par '_tags' ou '_en', en excluant 'countries_en'
redundant_columns = [col for col in df_cleaned.columns if (col.endswith('_tags') or col.endswith('_en')) and col != 'countries_en']

# Supprimer les colonnes redondantes du DataFrame
df_cleaned.drop(columns=redundant_columns, inplace=True)

# Afficher les colonnes après suppression
print("Colonnes restantes après suppression :")
print(df_cleaned.columns)


C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_6428\2082343226.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cleaned = pd.read_csv('nutriments_vides_cleaned.csv')


Colonnes restantes après suppression :
Index(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'last_modified_by',
       'last_updated_t', 'last_updated_datetime', 'product_name', 'quantity',
       'brands', 'categories', 'labels', 'countries', 'countries_en',
       'ingredients_text', 'serving_size', 'serving_quantity', 'additives_n',
       'nutriscore_score', 'nutriscore_grade', 'nova_group', 'pnns_groups_1',
       'pnns_groups_2', 'food_groups', 'states', 'ecoscore_score',
       'ecoscore_grade', 'product_quantity', 'unique_scans_n', 'completeness',
       'last_image_t', 'last_image_datetime', 'main_category', 'image_url',
       'image_small_url', 'image_ingredients_url',
       'image_ingredients_small_url', 'image_nutrition_url',
       'image_nutrition_small_url', 'energy-kcal_100g', 'energy_100g',
       'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g',
       'fiber_100g', 'proteins_100g'